In [ ]:
import os
import json
import geopandas as gpd
import numpy as np
import rasterio as rio
import rioxarray as rxr
from rasterstats import zonal_stats
import psycopg2
from sqlalchemy import create_engine

#import pandas as pd
#from rasterio.plot import show
#import matplotlib.pyplot as plt
#import pyproj
#from matplotlib import pyplot
#import shapely
#import folium
#from folium.plugins import MousePosition

In [ ]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
os.chdir(r'D:\iMMAP\proj\ASDC\data\flood_layer')

In [ ]:
pyproj.datadir.get_data_dir()

## Preping vector data

In [ ]:
#32641
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con).to_crs(epsg=32641)   # Load postgres table
admin['area_km2'] = admin['geom'].area.div(1000000)
print(admin['area_km2'].sum())
admin.crs

In [ ]:
#32642
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con).to_crs(epsg=32642)   # Load postgres table
admin['area_km2'] = admin['geom'].area.div(1000000)
print(admin['area_km2'].sum())
admin.crs

In [ ]:
#4979
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con).to_crs('+proj=aea +lat_1=29.5 +lat_2=42.5')   # Load postgres table
admin['area_km2'] = admin['geom'].area.div(1000000)
print(admin['area_km2'].sum())
admin.crs

In [ ]:
#+proj=aea +lat_1=29.5 +lat_2=42.5'
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con).to_crs('+proj=aea +lat_1=29.5 +lat_2=42.5')   # Load postgres table
admin['area_km2'] = admin['geom'].area.div(1000000)
print(admin['area_km2'].sum())
admin.crs

In [ ]:
#+proj=leac
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con).to_crs('+proj=leac')   # Load postgres table
admin['area_km2'] = admin['geom'].area.div(1000000)
print(admin['area_km2'].sum())
admin.crs

In [ ]:
#+proj=cea
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con).to_crs('+proj=cea')   # Load postgres table
admin['area_km2'] = admin['geom'].area.div(1000000)
print(admin['area_km2'].sum())
admin.crs

### Testing projection - Lambert Cylindrical Equal Area

In [ ]:
# Load from postgis
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con)
admin.crs

In [ ]:
# Convert to CEA
output_crs = '+proj=cea'
admin_proj_cea = admin.to_crs(output_crs)   # Load postgres table
admin_proj_cea.crs

In [ ]:
# Convert back to 4326
output_crs = 'epsg:4326'
admin_proj_4326 = admin_proj_cea.to_crs(output_crs)
admin_proj_4326.crs

In [ ]:
# Save as 4326 in db
admin_proj_4326.to_postgis('admin_test_proj_lambert_4326_test', con)

In [ ]:
# Save as CEA in db
admin_proj_cea.to_postgis('admin_test_proj_lambert_CEA_test', con)

In [ ]:
# Load 4326 version
admin_proj_4326_load = gpd.GeoDataFrame.from_postgis('SELECT * from admin_test_proj_lambert_4326_test', con) #.to_crs('+proj=cea')
admin_proj_4326_load.crs

In [ ]:
# Load CEA version   ### Unable to load from postgis when in CEA projection (loads fine in QGIS though)
#admin_proj_CEA_load = gpd.GeoDataFrame.from_postgis('SELECT * from admin_test_proj_lambert_CEA_test', con) #.to_crs('+proj=cea')
#admin_proj_CEA_load.crs

### Testing projection on raster

In [ ]:
# Open raster
path = r'D:\iMMAP\proj\ASDC\data\sett_workspace_v04\input\pop-sample.tif'
pop_rxr = rxr.open_rasterio(path, masked=True).squeeze() # Open with rxr, much easier to reproject

In [ ]:
# Check crs
pop_rxr.rio.crs

In [ ]:
# Reproject the data using the crs from another layer
pop_rxr = pop_rxr.rio.reproject(admin_proj_cea.crs)
pop_rxr.rio.crs

In [ ]:
#Save the raster to a new tif with the new crs
pop_rxr.rio.to_raster(r'D:\iMMAP\proj\ASDC\data\pop-test2.tif')

In [ ]:
#Open the new tif to check if it works
pop_rxr_2 = rxr.open_rasterio(r'D:\iMMAP\proj\ASDC\data\pop-test2.tif', masked=True).squeeze()
pop_rxr_2.rio.crs

In [ ]:
#Convert to 4326 and save again
pop_rxr_3 = pop_rxr_2.rio.reproject(admin_proj_4326_load.crs)
pop_rxr_3.rio.to_raster(r'D:\iMMAP\proj\ASDC\data\pop-test3.tif')